# CONSULTA BD

In [ ]:
import pandas as pd
import os

# Detectar entorno para Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# Definir la ruta base para los archivos en Google Drive
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'

# Cargar el DataFrame de hermanos desde la ruta correcta
df_hermanos = pd.read_csv(os.path.join(RUTA_BASE, "base_datos_hermanos.csv"))

df_hermanos[(df_hermanos['genero'] == 'M') & (df_hermanos['privilegio'] == 'ANC')]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,nombre_completo,genero,privilegio,activo,f_presidente,f_oracion,f_lectura,f_tesoros,f_perlas,f_vida,f_estudio_libro,f_lector_libro,f_discurso_est,f_demos,f_ayudante,observaciones
4,102,Juan Pablo Alfonso,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
13,11,James Cabieles,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
19,13,Cristian Calderón,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
29,58,William Diaz,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN
47,3,Cesar Hernandez,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
49,1,Orlando Hernandez,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
68,9,Ricardo Morales,M,ANC,1,1,1,0,1,1,1,1,1,0,0,0,NaN
82,23,Edwin Quintero,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN
86,27,Gustavo Ramirez,M,ANC,1,1,1,0,1,1,1,1,0,0,1,1,NaN
88,17,Juan Carlos Riaño,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN


# 📥 FASE 1 v5.0: EXTRACTOR WEB ROBUSTO (WOL)

In [ ]:
# =============================================================================
#  🕷️ FASE 1 v5.0: EXTRACTOR UNIVERSAL (CAZADOR AUTOMÁTICO)
# =============================================================================
#  MEJORAS:
#  1. Pide AÑO y MES al usuario (ya no hay que editar fechas a mano).
#  2. Calcula automáticamente los lunes de ese mes.
#  3. Mantiene la lógica "Cazador" para encontrar todas las partes (Maestros).
# =============================================================================

import requests
from bs4 import BeautifulSoup
import json
import re
import os
import calendar
from datetime import date

# RUTA BASE (Verifica que sea correcta)
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILE_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')

def limpiar_texto(t): return t.replace('\u00a0', ' ').strip() if t else ""

def obtener_html(url):
    try:
        r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=20)
        return BeautifulSoup(r.content, 'html.parser') if r.status_code == 200 else None
    except: return None

def generar_urls_lunes(anio, mes):
    """Genera las URLs de WOL para todos los lunes del mes dado"""
    urls = []
    cal = calendar.Calendar(firstweekday=calendar.MONDAY)
    for semana in cal.monthdatescalendar(anio, mes):
        lunes = semana[0]
        # Solo si el lunes pertenece al mes solicitado
        if lunes.month == mes:
            urls.append(f"https://wol.jw.org/es/wol/dt/r4/lp-s/{lunes.year}/{lunes.month}/{lunes.day}")
    return urls

def extractor_universal():
    print("🌍 GENERADOR DE PROGRAMA JW (Fase 1 v5.0)")
    print("=========================================")

    # INTERACCIÓN CON EL USUARIO
    try:
        anio_input = int(input("👉 Ingresa el AÑO (ej. 2026): "))
        mes_input = int(input("👉 Ingresa el MES (1=Ene, 2=Feb...): "))
    except ValueError:
        return print("❌ Error: Debes ingresar números válidos.")

    urls_a_procesar = generar_urls_lunes(anio_input, mes_input)
    print(f"\n   📅 Se detectaron {len(urls_a_procesar)} semanas para procesar.")

    datos = []

    for url in urls_a_procesar:
        print(f"\n   📥 Descargando: {url} ...")
        soup = obtener_html(url)
        if not soup:
            print("      ❌ Error de conexión.")
            continue

        # 1. INFO GENERAL
        elem_titulo = soup.find('h1', id='p1')
        txt_sem = limpiar_texto(elem_titulo.get_text()) if elem_titulo else "Semana desconocida"
        print(f"      🗓️ {txt_sem}")

        # CANCIONES
        canciones = []
        for c in soup.find_all(['p', 'div']): # A veces cambia el tag
             txt = c.get_text()
             if "Canción" in txt or "Cántico" in txt:
                 num = re.search(r'(\d+)', txt)
                 if num: canciones.append(num.group(1))

        # Lógica simple para asignar (Inicio, Medio, Final)
        c_ini = canciones[0] if len(canciones) > 0 else "---"
        c_med = canciones[1] if len(canciones) > 1 else "---"
        c_fin = canciones[2] if len(canciones) > 2 else "---"

        partes = []
        partes.append({"tipo": "Presidente", "titulo": "Presidente de la Reunión"})
        partes.append({"tipo": "Oracion_Inicio", "titulo": f"Canción {c_ini} y oración"})

        # 2. TESOROS
        tesoros_encontrado = False
        for elem in soup.find_all(['p', 'h3']):
            txt = limpiar_texto(elem.get_text())
            if "(10 mins.)" in txt and "Perlas" not in txt and "Consejo" not in txt:
                tit = txt.replace("(10 mins.)", "").strip()
                partes.append({"tipo": "Tesoros", "titulo": tit})
                tesoros_encontrado = True
                break

        if not tesoros_encontrado: partes.append({"tipo": "Tesoros", "titulo": "Discurso de Tesoros"})
        partes.append({"tipo": "Perlas", "titulo": "Busquemos Perlas Escondidas"})

        # LECTURA
        lectura_tit = "Lectura de la Biblia"
        for elem in soup.find_all(['p', 'h3']):
            if "(4 mins.)" in elem.get_text():
                match = re.search(r'\)\s*(.+)', elem.get_text())
                if match: lectura_tit = match.group(1).strip()
                break
        partes.append({"tipo": "Lectura", "titulo": lectura_tit})

        # 3. MAESTROS (Modo Cazador)
        claves = ["Empiece", "Haga revisitas", "Haga discípulos", "Explique", "Discurso"]
        h3s = soup.find_all('h3')
        count_m = 0
        for h3 in h3s:
            txt = limpiar_texto(h3.get_text())
            if "MAESTROS" in txt.upper() or "Estudio" in txt: continue

            if any(k in txt for k in claves):
                # Limpiar texto
                tit_clean = re.split(r'\(\d', txt)[0].strip().strip('.:0123456789')
                partes.append({"tipo": "Maestros", "titulo": tit_clean})
                count_m += 1
                print(f"         ✅ Maestro: {tit_clean}")

        if count_m == 0:
            print("         ⚠️ ALERTA: No se encontraron partes de Maestros. Verifica el HTML.")

        # 4. VIDA CRISTIANA (NVC)
        secc_vida = soup.find('div', id='section4')
        if secc_vida:
            for h3 in secc_vida.find_all('h3'):
                txt = limpiar_texto(h3.get_text())
                if any(x in txt for x in ["VIDA", "Cántico", "Oración", "Estudio", "Conclusión"]): continue
                tit = re.split(r'\(\d', txt)[0].strip()
                partes.append({"tipo": "NVC", "titulo": tit})

        # FIJOS FINALES
        titulo_ebc = "Estudio Bíblico de Congregación"
        # Intento extra de capturar título del libro
        for h3 in soup.find_all('h3'):
            if "Estudio bíblico" in h3.get_text():
                match = re.search(r'\)\s*(.+)', h3.get_text())
                if match: titulo_ebc = match.group(1).strip()

        partes.append({"tipo": "Estudio_Libro", "titulo": titulo_ebc})
        partes.append({"tipo": "Lector_Libro", "titulo": "Lectura del Estudio Bíblico"})
        partes.append({"tipo": "Oracion_Final", "titulo": f"Canción {c_fin} y oración"})

        datos.append({
            "semana": txt_sem,
            "cancion_inicio": c_ini,
            "cancion_medio": c_med,
            "cancion_final": c_fin,
            "partes": partes
        })

    # GUARDAR
    # Modo 'w' sobrescribe el archivo, perfecto para limpiar errores previos
    with open(FILE_JSON, 'w', encoding='utf-8') as f:
        json.dump(datos, f, indent=4, ensure_ascii=False)

    print(f"\n💾 ¡LISTO! JSON generado para {mes_input}/{anio_input}")
    print(f"   Archivo: {os.path.basename(FILE_JSON)}")

if __name__ == "__main__":
    extractor_universal()

🌍 GENERADOR DE PROGRAMA JW (Fase 1 v5.0)
👉 Ingresa el AÑO (ej. 2026): 2026
👉 Ingresa el MES (1=Ene, 2=Feb...): 1

   📅 Se detectaron 4 semanas para procesar.

   📥 Descargando: https://wol.jw.org/es/wol/dt/r4/lp-s/2026/1/5 ...
      🗓️ 5-11 DE ENERO
         ✅ Maestro:  Empiece conversaciones
         ✅ Maestro:  Haga revisitas
         ✅ Maestro:  Discurso

   📥 Descargando: https://wol.jw.org/es/wol/dt/r4/lp-s/2026/1/12 ...
      🗓️ 12-18 DE ENERO
         ✅ Maestro:  Empiece conversaciones
         ✅ Maestro:  Empiece conversaciones
         ✅ Maestro:  Haga revisitas
         ✅ Maestro:  Discurso

   📥 Descargando: https://wol.jw.org/es/wol/dt/r4/lp-s/2026/1/19 ...
      🗓️ 19-25 DE ENERO
         ✅ Maestro:  Empiece conversaciones
         ✅ Maestro:  Haga revisitas
         ✅ Maestro:  Explique sus creencias

   📥 Descargando: https://wol.jw.org/es/wol/dt/r4/lp-s/2026/1/26 ...
      🗓️ 26 DE ENERO A 1 DE FEBRERO
         ✅ Maestro:  Empiece conversaciones
         ✅ Maestro:  Hag

# 🦴 OLD_Fase2_Generador_Excel.py

In [ ]:
# =============================================================================
#  🤖 ASISTENTE DE PROGRAMACIÓN VMC - FASE 2: EXTRACCIÓN HÍBRIDA (Excel + JSON)
# =============================================================================
#  RAMA: fix-fase1
#  CAMBIOS:
#  1. Genera Excel (Legacy) para respaldo.
#  2. Genera JSON (Nuevo) para el motor de asignación automática.
#  3. Soporta Enero 2026 correctamente.
# =============================================================================

import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import calendar
import json
import time
import os

# Detectar entorno
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# CONFIGURACIÓN
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
RUTA_EXCEL = os.path.join(RUTA_BASE, 'Programación VMC_Septiembre-2025-2026.xlsx')
RUTA_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')

COLUMNAS_ORDENADAS = [
    'Semana', 'Libro', 'Canción Inicial', 'Tesoros de la Biblia',
    'Segunda Canción', 'Tercera Canción',
    'Maestros Título 1', 'Maestros Título 2', 'Maestros Título 3', 'Maestros Título 4',
    'NVC Título 1', 'NVC Título 2', 'NVC Título 3',
    'Info Lectura Biblia', 'Info Estudio Libro'
]

def obtener_fecha_interactiva():
    print("📅 CONFIGURACIÓN DE BÚSQUEDA")
    try:
        anio = int(input("👉 AÑO (ej. 2026): "))
        mes = int(input("👉 MES (1-12): "))
        return anio, mes
    except: return None, None

def generar_urls_dinamicas(anio, mes):
    print(f"\n🔍 Calculando semanas para: {mes}/{anio}...")
    urls = []
    c = calendar.Calendar(firstweekday=calendar.MONDAY)
    for week in c.monthdatescalendar(anio, mes):
        if week[0].month == mes:
            urls.append(f"https://wol.jw.org/es/wol/dt/r4/lp-s/{week[0].year}/{week[0].month:02d}/{week[0].day:02d}")
    return urls

def limpiar_texto_estudio(texto):
    if not texto: return ""
    libros = ['lfb', 'bt', 'lmd', 'lvs', 'cf', 'rr', 'ia', 'jr']
    for l in libros: texto = re.sub(rf"\b{l}(?=[a-zA-Z0-9])", f"{l} ", texto, flags=re.IGNORECASE)
    return texto

def extraer_informacion(url):
    try:
        soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).content, 'html.parser')
        info = {col: "" for col in COLUMNAS_ORDENADAS}

        # Básicos
        info['Semana'] = soup.find('h1', id='p1').get_text(strip=True) if soup.find('h1', id='p1') else "N/A"
        if soup.find('h2', id='p2'): info['Libro'] = soup.find('h2', id='p2').get_text(strip=True)

        # Tesoros (Lógica Refinada S-38: Discurso de 10 mins)
        for elem in soup.find_all(lambda t: t.name in ['h3','p'] and "(10 mins.)" in t.text):
            if "Perlas" not in elem.text:
                info['Tesoros de la Biblia'] = elem.get_text(strip=True).replace("(10 mins.)", "").strip()
                break

        # Canciones
        canciones = [f"Canción {re.search(r'(\d+)', h3.text).group(1)}" for h3 in soup.find_all('h3') if "Canción" in h3.text and re.search(r'(\d+)', h3.text)]
        if len(canciones) > 0: info['Canción Inicial'] = canciones[0]
        if len(canciones) > 1: info['Segunda Canción'] = canciones[1]
        if len(canciones) > 2: info['Tercera Canción'] = canciones[2]

        # Lectura y Estudio
        for p in soup.find_all('p'):
            if "(4 mins.)" in p.text and not info['Info Lectura Biblia']:
                info['Info Lectura Biblia'] = re.search(r'\((?:4|3)\s+mins?.*?\)\s*(.+)', p.text).group(1).strip()
            if "(30 mins.)" in p.text and not info['Info Estudio Libro']:
                info['Info Estudio Libro'] = limpiar_texto_estudio(re.search(r'\(30\s+mins?.*?\)\s*(.+)', p.text).group(1).strip())

        # Secciones Maestros y Vida
        seccion, titulos_maestros, nvc_titulos = None, [], []
        for h in soup.find_all(['h2', 'h3']):
            txt = h.get_text(strip=True)
            if h.name == 'h2':
                if "MEJORES MAESTROS" in txt.upper(): seccion = "MAESTROS"
                elif "VIDA CRISTIANA" in txt.upper(): seccion = "VIDA"
            elif h.name == 'h3' and seccion:
                if any(x in txt for x in ["Canción", "Conclusión", "Artículo"]): continue
                if "Artículo de estudio" in txt: break

                if seccion == "MAESTROS": titulos_maestros.append(txt)
                elif seccion == "VIDA" and "Estudio bíblico" not in txt: nvc_titulos.append(txt)

        for i, t in enumerate(titulos_maestros[:4]): info[f'Maestros Título {i+1}'] = t
        for i, t in enumerate(nvc_titulos[:3]): info[f'NVC Título {i+1}'] = t
        info['NVC Título 3'] = "Estudio bíblico de la congregación" # Default

        return info
    except Exception as e:
        print(f"❌ Error {url}: {e}")
        return None

def guardar_excel_limpio(datos):
    print(f"\n💾 (LEGACY) Guardando Excel: {os.path.basename(RUTA_EXCEL)}...")
    try:
        book = load_workbook(RUTA_EXCEL) if os.path.exists(RUTA_EXCEL) else pd.ExcelWriter(RUTA_EXCEL, engine='openpyxl').book
        if 'Info-reunion' not in book.sheetnames: book.create_sheet('Info-reunion')
        ws = book['Info-reunion']

        if ws.max_row > 1: ws.delete_rows(2, amount=ws.max_row-1) # Limpiar

        for r_idx, row in enumerate(dataframe_to_rows(pd.DataFrame(datos, columns=COLUMNAS_ORDENADAS), index=False, header=False), 1):
            for c_idx, val in enumerate(row, 1): ws.cell(row=r_idx+1, column=c_idx, value=val)

        book.save(RUTA_EXCEL)
        print("✅ Excel actualizado.")
    except Exception as e: print(f"❌ Error Excel: {e}")

# --- AQUÍ ESTÁ LA NUEVA FUNCIÓN JSON ---
def guardar_json_reuniones(datos):
    print(f"\n💾 (NUEVO) Generando JSON para Motor de Asignación...")
    data_estructurada = []

    for d in datos:
        # Aquí aplicamos lógica básica basada en S-38 para definir requisitos
        semana_obj = {
            "semana": d['Semana'],
            "partes": [
                {"tipo": "Presidente", "titulo": "Presidencia", "requiere": "ANC", "tiempo": 0},
                {"tipo": "Oracion_Inicio", "titulo": "Oración", "requiere": "H_BAUTIZADO", "tiempo": 5},
                {"tipo": "Tesoros", "titulo": d['Tesoros de la Biblia'], "requiere": "ANC_SM", "tiempo": 10},
                {"tipo": "Perlas", "titulo": "Perlas Escondidas", "requiere": "ANC_SM", "tiempo": 10},
                {"tipo": "Lectura", "titulo": d['Info Lectura Biblia'], "requiere": "EST_VARON", "tiempo": 4}
            ]
        }

        # Maestros (Dinámico)
        for i in range(1, 5):
            titulo = d.get(f'Maestros Título {i}')
            if titulo:
                # Análisis simple de texto para inferir si es Discurso (Solo varones)
                req = "ESTUDIANTE"
                if "Discurso" in titulo: req = "EST_VARON"
                semana_obj['partes'].append({"tipo": "Maestros", "titulo": titulo, "requiere": req, "tiempo": 5})

        # Vida Cristiana
        for i in range(1, 3):
            titulo = d.get(f'NVC Título {i}')
            if titulo:
                semana_obj['partes'].append({"tipo": "NVC", "titulo": titulo, "requiere": "ANC_SM", "tiempo": 15})

        semana_obj['partes'].extend([
            {"tipo": "Estudio_Libro", "titulo": "Estudio Bíblico", "requiere": "ANC_SM", "tiempo": 30},
            {"tipo": "Lector_Libro", "titulo": "Lectura EBC", "requiere": "H_BAUTIZADO", "tiempo": 0},
            {"tipo": "Oracion_Final", "titulo": "Oración", "requiere": "H_BAUTIZADO", "tiempo": 5}
        ])

        data_estructurada.append(semana_obj)

    with open(RUTA_JSON, 'w', encoding='utf-8') as f:
        json.dump(data_estructurada, f, indent=4, ensure_ascii=False)
    print(f"✅ JSON guardado en: {os.path.basename(RUTA_JSON)}")

# EJECUCIÓN
if __name__ == "__main__":
    anio, mes = obtener_fecha_interactiva()
    if anio:
        datos = [extraer_informacion(url) for url in generar_urls_dinamicas(anio, mes) if extraer_informacion(url)]
        if datos:
            guardar_excel_limpio(datos)      # Mantenemos el puente viejo
            guardar_json_reuniones(datos)    # Construimos el puente nuevo
            print("\n🎉 FASE 1 COMPLETADA (HÍBRIDA).")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📅 CONFIGURACIÓN DE BÚSQUEDA


# 🤖 FASE 2 v2.2: Generador de Asginaciones

In [3]:
# =============================================================================
#  🤖 FASE 3 v2.2: GENERADOR DE ASGINACIONES (Estructura POO + Habilidades)
# =============================================================================

import json
import pandas as pd
import os
from datetime import datetime, timedelta

try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# --- CONFIGURACIÓN ---
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'

FILES = {
    "DB_HERMANOS": os.path.join(RUTA_BASE, "base_datos_hermanos.csv"),
    "JSON_REUNIONES": os.path.join(RUTA_BASE, "data_reuniones_2026.json"),
    "HISTORIAL": os.path.join(RUTA_BASE, "historial_asignaciones.csv"),
    "OUTPUT_JSON": os.path.join(RUTA_BASE, "programa_completo_asignado.json")
}

# FECHA DE INICIO (Ajustar según el primer lunes de tu programación)
FECHA_INICIO_PROGRAMA = datetime(2026, 1, 5)

# MAPEO: Tipo de parte en JSON -> Columna de habilidad en CSV
# (Asegúrate que estas columnas existan en tu base_datos_hermanos.csv)
MAPEO_HABILIDADES = {
    'Presidente': 'f_presidente',
    'Oracion': 'f_oracion', # Si en JSON dice "Oracion_Inicio" o "Oracion_Final", usaremos contains
    'Tesoros': 'f_tesoros',
    'Perlas': 'f_perlas',
    'Lectura': 'f_lectura',
    'Maestros': 'f_demos',    # Genérico para estudiantes
    'Vida': 'f_vida',         # Partes varias de NVC
    'Estudio_Libro': 'f_estudio_libro', # Conductor
    'Lector_Libro': 'f_lector_libro'
}

class MotorHibridoV3:
    def __init__(self):
        self.db_hermanos = pd.DataFrame()
        self.historial = pd.DataFrame()
        self.data_reuniones = []
        self.cargar_datos()

    def cargar_datos(self):
        print("📥 INICIALIZANDO MOTOR V3.2 (Con validación de habilidades)...")

        # 1. DB Hermanos
        if os.path.exists(FILES["DB_HERMANOS"]):
            self.db_hermanos = pd.read_csv(FILES["DB_HERMANOS"])
            # Convertimos flags a numérico por seguridad
            cols_flags = [col for col in self.db_hermanos.columns if col.startswith('f_')]
            for col in cols_flags:
                self.db_hermanos[col] = pd.to_numeric(self.db_hermanos[col], errors='coerce').fillna(0)
            print(f"   ✅ BD Hermanos: {len(self.db_hermanos)} registros.")
        else:
            print("   ❌ ERROR CRÍTICO: No encuentro base_datos_hermanos.csv")

        # 2. Historial
        if os.path.exists(FILES["HISTORIAL"]):
            self.historial = pd.read_csv(FILES["HISTORIAL"])
            self.historial['fecha_asignacion'] = pd.to_datetime(self.historial['fecha_asignacion'])
            print(f"   ✅ Historial: {len(self.historial)} registros.")
        else:
            self.historial = pd.DataFrame(columns=['fecha_asignacion', 'id_hermano', 'tipo_asignacion'])

        # 3. JSON Reuniones
        if os.path.exists(FILES["JSON_REUNIONES"]):
            with open(FILES["JSON_REUNIONES"], 'r', encoding='utf-8') as f:
                self.data_reuniones = json.load(f)
            print(f"   ✅ JSON Cargado: {len(self.data_reuniones)} semanas.")
        else:
            print("   ❌ ERROR CRÍTICO: No encuentro el JSON de reuniones.")

    def verificar_manual(self, fecha_dt, tipo_parte):
        """Revisa si ya existe asignación en el historial."""
        if self.historial.empty: return None, None

        filtro = (self.historial['fecha_asignacion'] == fecha_dt) & \
                 (self.historial['tipo_asignacion'] == tipo_parte)

        match = self.historial[filtro]
        if not match.empty:
            return match.iloc[0]['nombre_hermano'], match.iloc[0]['id_hermano']
        return None, None

    def obtener_flag_habilidad(self, tipo_parte, titulo_parte=""):
        """Determina qué columna 'f_' revisar en el CSV."""
        # Lógica especial para Maestros
        if 'Maestros' in tipo_parte:
            if 'Ayudante' in titulo_parte: return 'f_ayudante'
            return 'f_demos' # Por defecto estudiante

        # Búsqueda general en el diccionario
        for key, flag in MAPEO_HABILIDADES.items():
            if key in tipo_parte:
                return flag

        return None # Si no encuentra (ej: Cancion), no filtra por habilidad

    def buscar_candidato_auto(self, tipo_parte, titulo_parte, ids_excluidos, genero_req=None):
        """Busca candidato filtrando por HABILIDAD + GÉNERO + DESCANSO"""

        # 1. Copia base
        candidatos = self.db_hermanos.copy()

        # 2. FILTRO DE HABILIDAD (¡Lo que faltaba!)
        flag = self.obtener_flag_habilidad(tipo_parte, titulo_parte)
        if flag and flag in candidatos.columns:
            candidatos = candidatos[candidatos[flag] == 1]
        elif flag:
            print(f"      ⚠️ Advertencia: Columna '{flag}' no existe en CSV. Ignorando filtro.")

        # 3. Filtro Género
        if genero_req:
            candidatos = candidatos[candidatos['genero'] == genero_req]

        # 4. Excluir IDs usados hoy
        candidatos = candidatos[~candidatos['id'].isin(ids_excluidos)]

        if candidatos.empty: return None, None

        # 5. Ranking LRU (Least Recently Used)
        ranking = []
        for _, h in candidatos.iterrows():
            hist_h = self.historial[self.historial['id_hermano'] == h['id']]

            if hist_h.empty:
                ultima = datetime(2020, 1, 1) # Prioridad máxima
            else:
                ultima = hist_h['fecha_asignacion'].max()

            ranking.append({'id': h['id'], 'nombre': h['nombre_completo'], 'ultima': ultima})

        # Ordenar: Más antiguos primero
        df_rank = pd.DataFrame(ranking).sort_values(by='ultima', ascending=True)

        if not df_rank.empty:
            # Selección aleatoria entre los Top 3 para variedad
            top_3 = df_rank.head(3)
            seleccionado = top_3.sample(n=1).iloc[0]
            return seleccionado['id'], seleccionado['nombre']

        return None, None

    def procesar(self):
        print("\n⚙️ EJECUTANDO ASIGNACIÓN INTELIGENTE (POO + Habilidades)...")

        SOLO_VARONES = ['Presidente', 'Oracion', 'Tesoros', 'Lectura', 'Estudio_Libro', 'Lector_Libro']

        programa_final = []

        for i, semana in enumerate(self.data_reuniones):
            semana_txt = semana.get('semana', f'Semana {i+1}')

            # Cálculo matemático de la fecha
            fecha_dt = FECHA_INICIO_PROGRAMA + timedelta(weeks=i)
            print(f"\n📅 Procesando: {semana_txt} (Fecha: {fecha_dt.date()})")

            partes_asignadas = []
            ids_usados_hoy = []

            # Pre-cargar manuales de hoy a la lista de excluidos
            manuales_hoy = self.historial[self.historial['fecha_asignacion'] == fecha_dt]
            ids_usados_hoy = manuales_hoy['id_hermano'].tolist()

            # Iterar sobre la LISTA de partes (Corrección clave del amigo)
            for parte in semana.get('partes', []):
                tipo = parte.get('tipo', 'Desconocido')
                titulo = parte.get('titulo', tipo)
                parte_new = parte.copy()

                # Ignorar items que no se asignan (Canciones, Videos)
                if 'Cancion' in tipo or 'Video' in tipo:
                    partes_asignadas.append(parte_new)
                    continue

                # A. CHECK MANUAL
                nom_manual, id_manual = self.verificar_manual(fecha_dt, tipo)

                if nom_manual:
                    parte_new['asignado_a'] = nom_manual
                    parte_new['origen'] = 'MANUAL'
                    print(f"   ✅ Manual: {tipo} -> {nom_manual}")
                else:
                    # B. AUTO FILL
                    # Determinar género
                    genero = 'M' if any(x in tipo for x in SOLO_VARONES) else None
                    if 'Maestros' in tipo and 'Discurso' in titulo: genero = 'M' # Discurso estudiante siempre varón

                    id_auto, nom_auto = self.buscar_candidato_auto(tipo, titulo, ids_usados_hoy, genero)

                    if nom_auto:
                        parte_new['asignado_a'] = nom_auto
                        parte_new['origen'] = 'AUTO'
                        print(f"   🤖 Auto: {tipo} ({titulo[:20]}...) -> {nom_auto}")

                        ids_usados_hoy.append(id_auto)
                        nuevo_reg = {
                            'fecha_asignacion': fecha_dt,
                            'id_hermano': id_auto,
                            'tipo_asignacion': tipo,
                            'nombre_hermano': nom_auto
                        }
                        self.historial = pd.concat([self.historial, pd.DataFrame([nuevo_reg])], ignore_index=True)
                    else:
                        parte_new['asignado_a'] = "PENDIENTE"
                        parte_new['origen'] = 'VACIO'
                        print(f"   ⚠️ Sin candidato: {tipo}")

                partes_asignadas.append(parte_new)

            semana_out = semana.copy()
            semana_out['partes'] = partes_asignadas
            programa_final.append(semana_out)

        self.data_reuniones = programa_final

    def guardar(self):
        with open(FILES["OUTPUT_JSON"], 'w', encoding='utf-8') as f:
            json.dump(self.data_reuniones, f, indent=4, ensure_ascii=False)
        print(f"\n💾 ¡LISTO! Archivo generado: {os.path.basename(FILES['OUTPUT_JSON'])}")

# EJECUCIÓN
if __name__ == "__main__":
    app = MotorHibridoV3()
    app.procesar()
    app.guardar()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📥 INICIALIZANDO MOTOR V3.2 (Con validación de habilidades)...
   ✅ BD Hermanos: 101 registros.
   ✅ Historial: 244 registros.
   ✅ JSON Cargado: 4 semanas.

⚙️ EJECUTANDO ASIGNACIÓN INTELIGENTE (POO + Habilidades)...

📅 Procesando: 5-11 DE ENERO (Fecha: 2026-01-05)
   🤖 Auto: Presidente (Presidente de la Reu...) -> Armando Toasura
   🤖 Auto: Oracion_Inicio (Canción 2026 y oraci...) -> Paul Garcia
   🤖 Auto: Tesoros (...) -> Juan Pablo Alfonso
   🤖 Auto: Perlas (Busquemos Perlas Esc...) -> Orlando Hernandez
   ✅ Manual: Lectura -> Duvan Torres
   ✅ Manual: Maestros -> Dora Mendoza
   ✅ Manual: Maestros -> Dora Mendoza
   ✅ Manual: Maestros -> Dora Mendoza
   🤖 Auto: Estudio_Libro (Estudio Bíblico de C...) -> Cesar Hernandez
   🤖 Auto: Lector_Libro (Lectura del Estudio ...) -> Cristian Calderón
   🤖 Auto: Oracion_Final (Canción 2026 y oraci...) -> Manuel Césped

# ✍️ Fase 2 v4.0: EDITOR MAESTRO

In [ ]:
# =============================================================================
#  ✏️ FASE 3 (FINAL): EDITOR MAESTRO VMC (Con Ausencias + Edición JSON)
# =============================================================================
import pandas as pd
import json
import os
from datetime import datetime, timedelta

# --- RUTAS ---
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILES = {
    "DB": os.path.join(RUTA_BASE, 'base_datos_hermanos.csv'),
    "HIST": os.path.join(RUTA_BASE, 'historial_asignaciones.csv'),
    "JSON_OUT": os.path.join(RUTA_BASE, 'programa_completo_asignado.json'), # Lee el YA asignado
    "AUSENCIAS": os.path.join(RUTA_BASE, 'ausencias.csv')
}
FECHA_BASE = datetime(2026, 1, 5) # Ajustar primer lunes del programa

# MAPEO HABILIDADES
MAPEO = {
    'Presidente': 'f_presidente', 'Oracion': 'f_oracion', 'Tesoros': 'f_tesoros',
    'Perlas': 'f_perlas', 'Lectura': 'f_lectura', 'Maestros': 'f_demos',
    'Vida': 'f_vida', 'Estudio_Libro': 'f_estudio_libro', 'Lector_Libro': 'f_lector_libro',
    'Ayudante': 'f_ayudante'
}
SOLO_VARONES = ['Presidente', 'Oracion', 'Tesoros', 'Lectura', 'Estudio_Libro', 'Lector_Libro']

class CerebroTeocratico:
    def __init__(self):
        self.db = pd.DataFrame()
        self.hist = pd.DataFrame()
        self.ausencias = pd.DataFrame()
        self.cargar_datos()

    def cargar_datos(self):
        # 1. Base Hermanos
        if os.path.exists(FILES["DB"]):
            self.db = pd.read_csv(FILES["DB"])
            for c in [x for x in self.db.columns if x.startswith('f_')]:
                self.db[c] = pd.to_numeric(self.db[c], errors='coerce').fillna(0)

        # 2. Historial
        if os.path.exists(FILES["HIST"]):
            self.hist = pd.read_csv(FILES["HIST"])
            self.hist['fecha_asignacion'] = pd.to_datetime(self.hist['fecha_asignacion'])
        else:
            self.hist = pd.DataFrame(columns=['fecha_asignacion', 'id_hermano', 'nombre_hermano', 'tipo_asignacion'])

        # 3. Ausencias (NUEVO)
        if os.path.exists(FILES["AUSENCIAS"]):
            self.ausencias = pd.read_csv(FILES["AUSENCIAS"])
            # Convertir fechas
            self.ausencias['fecha_inicio'] = pd.to_datetime(self.ausencias['fecha_inicio'], errors='coerce')
            self.ausencias['fecha_fin'] = pd.to_datetime(self.ausencias['fecha_fin'], errors='coerce')
        else:
            self.ausencias = pd.DataFrame(columns=['id_hermano', 'fecha_inicio', 'fecha_fin'])

    def esta_ausente(self, id_hermano, fecha_reunion):
        """Verifica si la fecha cae en rango de ausencia."""
        if self.ausencias.empty: return False

        filtro = (self.ausencias['id_hermano'] == id_hermano) & \
                 (self.ausencias['fecha_inicio'] <= fecha_reunion) & \
                 (self.ausencias['fecha_fin'] >= fecha_reunion)

        return not self.ausencias[filtro].empty

    def obtener_sugerencias(self, tipo, titulo, fecha_dt, ids_excluidos=[]):
        candidatos = self.db[self.db['activo'] == 1].copy()

        # A. Filtro Habilidad
        flag = 'f_demos'
        if 'Ayudante' in tipo: flag = 'f_ayudante'
        else:
            for k, v in MAPEO.items():
                if k in tipo: flag = v; break

        if flag in candidatos.columns: candidatos = candidatos[candidatos[flag] == 1]

        # B. Filtro Género
        es_varon = any(x in tipo for x in SOLO_VARONES)
        if 'Maestros' in tipo and 'Discurso' in titulo: es_varon = True
        if es_varon: candidatos = candidatos[candidatos['genero'] == 'M']

        # C. Filtro Ausencia (NUEVO)
        candidatos['ausente'] = candidatos['id'].apply(lambda x: self.esta_ausente(x, fecha_dt))
        candidatos = candidatos[candidatos['ausente'] == False]

        # D. Exclusión y Ranking
        candidatos = candidatos[~candidatos['id'].isin(ids_excluidos)]

        ranking = []
        for _, h in candidatos.iterrows():
            hist_h = self.hist[self.hist['id_hermano'] == h['id']]
            if hist_h.empty:
                dias = 999
            else:
                ultima = hist_h['fecha_asignacion'].max()
                dias = (datetime.now() - ultima).days
            ranking.append({'id': h['id'], 'nombre': h['nombre_completo'], 'dias': dias})

        return pd.DataFrame(ranking).sort_values(by='dias', ascending=False).head(5)

# --- INTERFAZ DE EDICIÓN ---
def editor_maestro():
    print("📝 EDITOR MAESTRO VMC (Edita JSON + Guarda Historial)")

    cerebro = CerebroTeocratico()

    # Cargar JSON generado por Fase 2 (Auto)
    if not os.path.exists(FILES["JSON_OUT"]):
        return print("❌ No existe 'programa_completo_asignado.json'. Corre primero el Generador (Fase 2).")

    with open(FILES["JSON_OUT"], 'r', encoding='utf-8') as f:
        programa = json.load(f)

    # SELECCIÓN DE SEMANA
    print("\n📅 SEMANAS DISPONIBLES:")
    for i, s in enumerate(programa):
        print(f"   {i+1}. {s['semana']}")

    try:
        idx = int(input("\n👉 Elige Semana: ")) - 1
        semana = programa[idx]
    except: return

    fecha_sem = FECHA_BASE + timedelta(weeks=idx)
    print(f"\n🔧 Editando: {semana['semana']} (Fecha ref: {fecha_sem.date()})")

    # EDICIÓN
    cambios_json = False
    nuevos_historial = []

    for parte in semana['partes']:
        tipo = parte['tipo']
        titulo = parte.get('titulo', tipo)
        asignado = parte.get('asignado_a', 'VACÍO')
        origen = parte.get('origen', 'MANUAL')

        print(f"\n🔹 {titulo} ({tipo})")
        print(f"   👤 ASIGNADO ACTUAL: {asignado} [{origen}]")

        accion = input("   👉 ENTER mantener | '?' sugerir | Nombre buscar: ").strip()

        if not accion: continue # Mantener actual

        id_sel, nom_sel = None, None

        # MODO SUGERENCIA
        if accion == '?':
            sugs = cerebro.obtener_sugerencias(tipo, titulo, fecha_sem)
            if sugs.empty:
                print("      ⚠️ Nadie disponible (Revisar ausencias/requisitos).")
            else:
                print(f"      {'ID':<3} {'NOMBRE':<25} {'DESCANSÓ'}")
                lista = sugs.values.tolist()
                for i, h in enumerate(lista): print(f"      [{i}] {h[1]:<25} {h[2]} días")

                sel = input("      Selecciona #: ")
                if sel.isdigit() and int(sel) < len(lista):
                    id_sel, nom_sel = lista[int(sel)][0], lista[int(sel)][1]

        # MODO BÚSQUEDA
        else:
            busq = cerebro.db[cerebro.db['nombre_completo'].str.contains(accion, case=False)]
            if not busq.empty:
                lista = busq.values.tolist()
                for i, h in enumerate(lista):
                    # Ajustar índices según tu CSV: id=0, nombre=1 (revisar tu estructura real)
                    # Basado en tu snippet: id, nombre_completo, genero...
                    print(f"      [{i}] {h[1]}") # h[1] es nombre_completo
                sel = input("      Selecciona #: ")
                if sel.isdigit():
                    # Como es dataframe, usamos iloc sobre el filtrado
                    fila = busq.iloc[int(sel)]
                    id_sel, nom_sel = fila['id'], fila['nombre_completo']

        # APLICAR CAMBIO
        if id_sel:
            # 1. Actualizar JSON en memoria
            parte['asignado_a'] = nom_sel
            parte['origen'] = 'MANUAL_EDITOR'
            cambios_json = True

            # 2. Preparar Historial
            nuevos_historial.append({
                'fecha_asignacion': fecha_sem.strftime('%Y-%m-%d'),
                'id_hermano': id_sel,
                'nombre_hermano': nom_sel,
                'tipo_asignacion': tipo
            })
            print(f"   ✅ Cambiado a: {nom_sel}")

            # Lógica Ayudante (Simplificada)
            if 'Maestros' in tipo and 'Discurso' not in titulo:
                # Preguntar ayudante si cambió el estudiante
                print("      ¿Cambiar Ayudante también?")
                # ... (Lógica similar para ayudante si se desea)

    # GUARDADO FINAL
    if cambios_json:
        print("\n💾 GUARDANDO CAMBIOS...")

        # A. Guardar JSON
        with open(FILES["JSON_OUT"], 'w', encoding='utf-8') as f:
            json.dump(programa, f, indent=4, ensure_ascii=False)
        print("   ✅ Archivo JSON actualizado (Listo para PDF).")

        # B. Guardar Historial
        if nuevos_historial:
            df_new = pd.DataFrame(nuevos_historial)
            # Agregar cabecera solo si no existe
            hdr = not os.path.exists(FILES["HIST"])
            df_new.to_csv(FILES["HIST"], mode='a', header=hdr, index=False)
            print("   ✅ Historial actualizado (Memoria futura).")
    else:
        print("\n👋 Saliste sin hacer cambios.")

if __name__ == "__main__":
    editor_maestro()

🕵️ REGISTRADOR CON DIAGNÓSTICO (v3.6)

📅 ELIGE LA SEMANA:
   1. 5-11 DE ENERO
   2. 12-18 DE ENERO
   3. 19-25 DE ENERO
   4. 26 DE ENERO A 1 DE FEBRERO

👉 Número: 1

   1. Solo Estudiantes | 2. Todo el programa
   👉 Opción: 1

📋 PLAN DE TRABAJO PARA: 5-11 DE ENERO
   Se detectaron 4 partes en el archivo JSON:
   1. Is 19:1-12 (th lección 11).
   2.  Empiece conversaciones
   3.  Haga revisitas
   4.  Discurso

⚠️ IMPORTANTE: Si faltan partes en la lista de arriba,
   es porque el JSON está incompleto. Deberás correr la FASE 1 de nuevo.

👉 Presiona ENTER para empezar a registrar...

🔹 Is 19:1-12 (th lección 11).
   ✅ YA TIENE ASIGNADO A: Duvan Torres
      ❓ ¿Cambiar a Duvan Torres? (ENTER para saltar)
         Escribe Nombre (o ENTER para saltar): 
   ⏩ Saltando a la siguiente parte...

🔹  Empiece conversaciones
      ❓ ¿Quién?
         Escribe Nombre (o ENTER para saltar): Dora
         [0] Dora Mendoza (F)
         👉 Elige #: 0
   💾 Registrado para guardar: Dora Mendoza
      ❓ ¿Qui

# Utilidad 1: Crear archivo de Ausencias


In [ ]:
# =============================================================================
#  🛠️ UTILIDAD: INICIALIZAR AUSENCIAS (CORREGIDO)
# =============================================================================
#  Crea el archivo ausencias.csv manejando la conexión a Google Drive.
# =============================================================================

import pandas as pd
import os
from google.colab import drive # 1. Importamos la librería de conexión

def inicializar_ausencias():
    print("🚑 INICIALIZANDO SISTEMA DE AUSENCIAS")
    print("-------------------------------------")

    # 2. VERIFICACIÓN DE CONEXIÓN (El paso clave que faltaba)
    if not os.path.exists('/content/drive'):
        print("🔌 Conectando a Google Drive...")
        drive.mount('/content/drive')
    else:
        print("✅ Google Drive ya está conectado.")

    # 3. RUTAS
    # Verifica que esta ruta sea la correcta (sin espacios extraños)
    RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
    FILE_AUSENCIAS = os.path.join(RUTA_BASE, 'ausencias.csv')

    # Validar que la carpeta madre exista
    if not os.path.exists(RUTA_BASE):
        print(f"❌ ERROR: No encuentro la carpeta del proyecto en:")
        print(f"   {RUTA_BASE}")
        return

    # 4. CREACIÓN DEL ARCHIVO
    if not os.path.exists(FILE_AUSENCIAS):
        # Columnas estándar para gestionar vacaciones
        cols = ['id_hermano', 'nombre', 'fecha_inicio', 'fecha_fin', 'motivo']

        df = pd.DataFrame(columns=cols)
        df.to_csv(FILE_AUSENCIAS, index=False, encoding='utf-8')

        print(f"\n✅ ¡ÉXITO! Archivo creado: {os.path.basename(FILE_AUSENCIAS)}")
        print("   Ahora puedes registrar las fechas en que los hermanos no estarán.")
    else:
        print(f"\n⚠️ El archivo ya existe en: {os.path.basename(FILE_AUSENCIAS)}")
        print("   No se ha borrado nada.")

if __name__ == "__main__":
    inicializar_ausencias()

🚑 INICIALIZANDO SISTEMA DE AUSENCIAS
-------------------------------------
✅ Google Drive ya está conectado.

✅ ¡ÉXITO! Archivo creado: ausencias.csv
   Ahora puedes registrar las fechas en que los hermanos no estarán.


# Uilidad 2: Agregar hermano nuevo

In [ ]:
# =============================================================================
#  🛠️ GESTIÓN DE HERMANOS: AGREGAR NUEVO (SIN ROMPER IDs)
# =============================================================================

import pandas as pd
import os

RUTA_BD = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/base_datos_hermanos.csv'

def agregar_hermano_manual():
    print("👤 INSCRIPCIÓN DE NUEVO HERMANO")
    print("-------------------------------")

    # 1. Cargar base de datos actual
    if not os.path.exists(RUTA_BD):
        print("❌ Error: No existe la base de datos.")
        return

    df = pd.read_csv(RUTA_BD)

    # 2. Calcular ID Automático (Max ID + 1) para integridad
    # Si la tabla está vacía, empieza en 1. Si no, busca el máximo.
    nuevo_id = 1
    if not df.empty:
        nuevo_id = df['id'].max() + 1

    print(f"🆔 ID Asignado Automáticamente: {nuevo_id}")

    # 3. Pedir datos
    nombre = input("Nombre Completo: ").strip().title()
    genero = input("Género (M/F): ").strip().upper()
    privilegio = input("Privilegio (PUB/SM/ANC): ").strip().upper()

    # Capacidades básicas (Preguntas rápidas)
    print("\n--- Capacidades (S/N) ---")
    lectura = 1 if input("¿Hace Lectura de Biblia? ").upper() == 'S' else 0
    demos = 1 if input("¿Hace Demostraciones/Ayudante? ").upper() == 'S' else 0
    discurso = 1 if input("¿Hace Discursos Estudiantiles? ").upper() == 'S' else 0

    # 4. Crear el diccionario con la estructura EXACTA
    nuevo_hermano = {
        'id': nuevo_id,
        'nombre_completo': nombre,
        'genero': genero,
        'privilegio': privilegio,
        'activo': 1,
        # Banderas
        'f_presidente': 0, 'f_oracion': 0,
        'f_lectura': lectura,
        'f_tesoros': 0, 'f_perlas': 0, 'f_vida': 0,
        'f_estudio_libro': 0, 'f_lector_libro': 0,
        'f_discurso_est': discurso,
        'f_demos': demos,
        'f_ayudante': demos, # Generalmente si hace demos, es ayudante
        'observaciones': "Ingreso manual"
    }

    # 5. Guardar
    # Convertimos el diccionario a DataFrame y lo concatenamos
    nuevo_df = pd.DataFrame([nuevo_hermano])

    # Alineamos columnas (para evitar desorden si faltan claves)
    # Usamos las columnas del df original como molde
    for col in df.columns:
        if col not in nuevo_df.columns:
            nuevo_df[col] = 0 # Rellenar con 0 lo que no preguntamos

    df_final = pd.concat([df, nuevo_df], ignore_index=True)
    df_final.to_csv(RUTA_BD, index=False, encoding='utf-8-sig')

    print(f"\n✅ ¡Listo! {nombre} ha sido inscrito con el ID {nuevo_id}.")

# Para usarlo, simplemente ejecuta:
# agregar_hermano_manual()

# Utilidad 3: MIGRADOR DE HISTORIAL (LEGACY -> SISTEMA NUEVO)

In [ ]:
# =============================================================================
#  🕰️ MIGRADOR DE HISTORIAL v3.0 (RUTA FORZADA)
# =============================================================================

import pandas as pd
import os
import re
from datetime import datetime

# --- CONFIGURACIÓN ---
# Usamos la misma ruta que ya sabemos que funciona para el JSON
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'

# PEGA AQUÍ EL NOMBRE EXACTO QUE SALIÓ EN EL DIAGNÓSTICO
NOMBRE_HISTORICO = 'historico_excel_asginaciones.csv'
NOMBRE_DB = 'base_datos_hermanos.csv'
NOMBRE_SALIDA = 'historial_asignaciones.csv'

# --- MAPEO (Tu diccionario de traducción) ---
MAPEO_COLUMNAS = {
    'PRESIDENCIA': 'Presidente',
    'ORACIÓN': 'Oracion_Inicio',
    'TESOROS DE LA BIBLIA': 'Tesoros',
    'BUSQUEMOS PERLAS ESCONDIDAS': 'Perlas',
    'LECTURA DE LA BIBLIA': 'Lectura',
    'SMM ASIG 1 ESTUD': 'Maestros', 'SMM ASIG 1 ACOMP': 'Ayudante',
    'SMM ASIG 2 ESTUD': 'Maestros', 'SMM ASIG 2 ACOMP': 'Ayudante',
    'SMM ASIG 3 ESTUD': 'Maestros', 'SMM ASIG 3 ACOMP': 'Ayudante',
    'SMM ASIG 4 ESTUD': 'Maestros', 'SMM ASIG 4 ACOMP': 'Ayudante',
    'NVC PARTE 1': 'NVC', 'NVC PARTE 2': 'NVC',
    'ESTUDIO LIBRO': 'Estudio_Libro', 'LECTOR LIBRO': 'Lector_Libro',
    'ORACIÓN FINAL': 'Oracion_Final'
}

def buscar_id_hermano(nombre_excel, df_db):
    if pd.isna(nombre_excel) or str(nombre_excel).strip() in ['', 'nan']: return None, None
    nombre_limpio = str(nombre_excel).strip()

    # 1. Exacto
    res = df_db[df_db['nombre_completo'].str.lower() == nombre_limpio.lower()]
    if not res.empty: return res.iloc[0]['id'], res.iloc[0]['nombre_completo']

    # 2. Parcial
    res = df_db[df_db['nombre_completo'].str.contains(nombre_limpio, case=False, regex=False)]
    if not res.empty: return res.iloc[0]['id'], res.iloc[0]['nombre_completo']

    return None, None

def extraer_fecha_lunes(texto_semana, mes_texto):
    try:
        dia = int(re.search(r'(\d+)', str(texto_semana)).group(1))
        mes_map = {'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12, 'ENERO': 1, 'FEBRERO': 2}
        mes_num = mes_map.get(str(mes_texto).upper().strip(), 1)
        anio = 2026 if mes_num < 9 else 2025
        return f"{anio}-{mes_num:02d}-{dia:02d}"
    except:
        return datetime.now().strftime("%Y-%m-%d")

def ejecutar_migracion():
    print("🚀 INICIANDO MIGRACIÓN v3.0")

    # Construir rutas completas
    path_old = os.path.join(RUTA_BASE, NOMBRE_HISTORICO)
    path_db = os.path.join(RUTA_BASE, NOMBRE_DB)
    path_out = os.path.join(RUTA_BASE, NOMBRE_SALIDA)

    # Verificación estricta
    if not os.path.exists(path_old):
        print(f"❌ ERROR: Python NO ve el archivo en: {path_old}")
        print("   Por favor ejecuta el script de Diagnóstico primero.")
        return

    print(f"   ✅ Archivo encontrado: {NOMBRE_HISTORICO}")

    df_old = pd.read_csv(path_old)
    df_db = pd.read_csv(path_db)

    nuevos = []

    for _, fila in df_old.iterrows():
        semana = fila.get('SEMANA', 'Semana X')
        fecha = extraer_fecha_lunes(semana, fila.get('MES', ''))

        for col_orig, tipo_dest in MAPEO_COLUMNAS.items():
            if col_orig in df_old.columns:
                nombre = fila[col_orig]
                id_h, nombre_real = buscar_id_hermano(nombre, df_db)

                if id_h:
                    nuevos.append({
                        'fecha_asignacion': fecha,
                        'semana_texto': semana,
                        'id_hermano': id_h,
                        'nombre_hermano': nombre_real,
                        'tipo_asignacion': tipo_dest,
                        'sala': 'Principal'
                    })

    if nuevos:
        df_migrado = pd.DataFrame(nuevos)
        # Modo 'append' seguro: Si existe historial, lo carga y añade
        if os.path.exists(path_out):
            df_existente = pd.read_csv(path_out)
            df_final = pd.concat([df_existente, df_migrado], ignore_index=True)
        else:
            df_final = df_migrado

        df_final.drop_duplicates(subset=['semana_texto', 'tipo_asignacion', 'nombre_hermano'], inplace=True)
        df_final.to_csv(path_out, index=False, encoding='utf-8')

        print(f"\n💾 ¡MIGRACIÓN EXITOSA!")
        print(f"   Se agregaron {len(nuevos)} registros históricos.")
        print(f"   Ahora el sistema conoce tu historial desde Septiembre.")
    else:
        print("⚠️ No se generaron registros. Revisa los nombres de las columnas en tu CSV.")

if __name__ == "__main__":
    ejecutar_migracion()

🚀 INICIANDO MIGRACIÓN v3.0
   ✅ Archivo encontrado: historico_excel_asginaciones.csv

💾 ¡MIGRACIÓN EXITOSA!
   Se agregaron 238 registros históricos.
   Ahora el sistema conoce tu historial desde Septiembre.


# Previo: Inicializar la Memoria (Historial Asginaciones)

In [ ]:
# =============================================================================
#  🛠️ UTILIDAD: INICIALIZAR HISTORIAL (CON AUTO-CONEXIÓN DRIVE)
# =============================================================================

import pandas as pd
import os
from google.colab import drive # Importamos la librería de Drive

def inicializar_historial():
    # --- PASO 1: VERIFICAR CONEXIÓN A DRIVE ---
    if not os.path.exists('/content/drive'):
        print("🔌 Conectando a Google Drive...")
        drive.mount('/content/drive')
    else:
        print("✅ Google Drive ya está conectado.")

    # --- PASO 2: CONFIGURACIÓN DE RUTA ---
    # Verifica que esta ruta sea EXACTAMENTE la de tu carpeta
    RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
    RUTA_HISTORIAL = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')

    # Verificar si la carpeta existe antes de intentar guardar
    if not os.path.exists(RUTA_BASE):
        print(f"❌ ERROR CRÍTICO: No encuentro la carpeta en Drive.")
        print(f"   Buscaba: {RUTA_BASE}")
        print("   👉 Verifica si el nombre de la carpeta tiene espacios o tildes diferentes.")
        return

    print(f"\n📂 Verificando historial en: {os.path.basename(RUTA_HISTORIAL)}")

    if os.path.exists(RUTA_HISTORIAL):
        print(f"⚠️ El archivo ya existe. No se ha borrado nada por seguridad.")
    else:
        columnas_maestras = [
            'fecha_asignacion', 'semana_texto', 'id_hermano',
            'nombre_hermano', 'tipo_asignacion', 'sala'
        ]

        df_vacio = pd.DataFrame(columns=columnas_maestras)
        df_vacio.to_csv(RUTA_HISTORIAL, index=False, encoding='utf-8')
        print(f"✅ ¡ÉXITO! Archivo de historial creado y listo.")

if __name__ == "__main__":
    inicializar_historial()

🔌 Conectando a Google Drive...
Mounted at /content/drive

📂 Verificando historial en: historial_asignaciones.csv
✅ ¡ÉXITO! Archivo de historial creado y listo.
